Primal problem:
$$\min c^T x \quad\text{subject to}\quad Ax=b, -x\leq 0.$$

Dual problem:
$$\max b^T y \quad\text{subject to}\quad A^Ty\leq c.$$

In [8]:
import numpy as np

In [9]:
# m: number of constraints
# n: number of variables
m = 50
n = 100

A = np.random.randn(m, n)
x0 = np.random.random(n)
b = A.dot(x0)
z = np.random.randn(m)
s = 10*np.random.random(n)

# This is to make sure that the dual problem is feasible in interior since A.T @ z < c.
c = A.T.dot(z) + s

In [10]:
print(x0)

[0.71293023 0.89826209 0.51418545 0.89813222 0.41482105 0.32036766
 0.49077246 0.41719171 0.88785186 0.04132433 0.05180415 0.76038737
 0.50309949 0.26521362 0.34107609 0.56734863 0.75872426 0.2493983
 0.00477283 0.28783023 0.19028056 0.81455969 0.6609955  0.12235089
 0.59266862 0.1112112  0.6524975  0.70844742 0.04315402 0.5557615
 0.67521984 0.32299484 0.00350825 0.25640881 0.08367344 0.93809604
 0.76658297 0.15578559 0.23349594 0.5418264  0.35956976 0.76642714
 0.69792854 0.12338252 0.00351007 0.5702551  0.83023413 0.65975707
 0.77884686 0.71583923 0.54672609 0.1056174  0.5269637  0.70364762
 0.78462248 0.56758224 0.78694155 0.93886385 0.00818485 0.41740253
 0.08635118 0.00868157 0.69975535 0.36280831 0.70516241 0.84408403
 0.83486425 0.15132324 0.12141933 0.48599298 0.33905536 0.84772746
 0.95256415 0.46918491 0.14372483 0.08550153 0.51146232 0.13708524
 0.5600333  0.96388544 0.58948538 0.5714311  0.75239809 0.00712361
 0.16105316 0.52881675 0.78356953 0.81751609 0.11497392 0.242875

In [11]:
def get_res(x, lambda_, v, tinv):
    r_dual = Df.dot(lambda_) + A.T.dot(v)+c
    r_cent = -np.diag(lambda_)@f(x) - tinv
    r_pri = A.dot(x)-b
    return r_dual, r_cent, r_pri

def get_res_norm(x, lambda_, v, tinv):
    r_dual, r_cent, r_pri = get_res(x, lambda_, v, tinv)
    return np.linalg.norm(r_dual), np.linalg.norm(r_cent), np.linalg.norm(r_pri)

In [12]:
MAXITERS = 500
TOL = 1e-8
RESTOL = 1e-8
mu = 10
alpha = 0.01
beta = 0.5
gaps = []
resdls = []
x = x0
f = lambda x: -x
Df = -np.eye(n)
lambda_ = -1 / (f(x))
v = np.zeros(m)

for iters in range(MAXITERS):
    # Surrogate duality gap
    ita = - f(x).dot(lambda_)
    gaps.append(ita)
    tinv = ita/(m*mu)
    # residual
    # r_dual = Df.dot(lambda_) + A.T.dot(v)+c
    # r_cent = -np.diag(lambda_)@f(x) - tinv
    # r_pri = A.dot(x)-b
    r_dual, r_cent, r_pri = get_res(x, lambda_, v, tinv)
    resdls.append(np.linalg.norm(np.hstack([r_dual, r_pri])))
    # stopping criterion
    if ita < TOL and np.linalg.norm(np.hstack([r_dual, r_pri])) < RESTOL:
        break
    
    sol = -np.linalg.solve(
        np.block([[np.zeros((n, n)), Df.T, A.T],
                  [-np.diag(lambda_)@Df, -np.diag(f(x)), np.zeros((n, m))],
                  [A, np.zeros((m, m)), np.zeros((m, n))]]),
        np.hstack([r_dual, r_cent, r_pri])
    )
    dx = sol[:n]
    dlambda_ = sol[n:n+n]
    dv = sol[-m:]

    # backtracking line search
    # The maximum step such that the new lambda_ is positive, i.e., feasible.
    step = min(1, 0.99/np.max(-dlambda_/lambda_))
    while True:
        x_new = x + step*dx
        if np.all(f(x_new) < 0):
            break
        step *= beta

    new_x = x + step*dx
    new_lambda_ = lambda_ + step*dlambda_
    new_v = v + step*dv
    
    # Old residual norm
    old_norm = sum(get_res_norm(x, lambda_, v, tinv))

    while sum(get_res_norm(new_x, new_lambda_, new_v, tinv)) > (1- alpha * step) * old_norm:
        step *= beta
        new_x = x + step*dx
        new_lambda_ = lambda_ + step*dlambda_
        new_v = v + step*dv

    x = new_x
    lambda_ = new_lambda_
    v = new_v

print(gaps[-1], resdls[-1], iters)
    

3.830031270967646e-09 2.661453620111205e-14 39


In [13]:
print(x)

[6.64818558e-01 6.87000730e-01 2.32406990e-01 6.81042780e-12
 4.44717322e-12 7.96416306e-12 3.75265599e-12 1.41316970e+00
 6.86604197e-12 2.45712829e-11 5.05658561e-01 1.89590843e-09
 8.60077572e-12 1.21763324e-11 9.88800701e-12 2.60482527e-01
 1.03174402e+00 2.83118401e-01 7.66245168e-12 1.68015457e-11
 4.97303184e-12 1.76164404e+00 8.07585949e-12 2.44719292e-10
 2.54438355e-01 7.39705349e-01 4.36375993e-01 2.19739476e-01
 2.23641645e-01 3.17081259e-11 5.52666059e-01 3.08596506e-12
 3.59677645e-12 3.77205165e-01 2.19272630e-11 1.42440477e+00
 3.84041447e-01 7.87699509e-01 9.82838792e-02 1.42240595e+00
 1.11121224e+00 1.19395524e-11 6.56933604e-01 7.96874216e-01
 4.15727300e-01 3.50290245e-12 4.89852083e-01 1.41782357e-11
 2.51188252e+00 1.28409147e-01 6.47446536e-12 1.06135345e-11
 9.25179824e-10 1.39963377e-11 4.36362210e-01 4.41601716e-12
 1.30009185e+00 2.17071333e-01 8.41417721e-12 2.03836555e-11
 3.67622377e-12 2.78109130e-12 9.83666271e-02 6.50288464e-01
 1.01999192e-01 7.026636

## Solve by Log-barrier method
In this case, we have the problem
$$\min t c^T x-log(x)\quad \text{subject to}\quad  Ax=b$$

In [14]:
from newton import newton_eq


In [15]:
t = 10
x_inner = x0.copy()
while m/t > TOL:
    f = lambda x: t*c.dot(x) - np.sum(np.log(x))
    grad_f = lambda x: t*c - 1/x
    nabla_f = lambda x: np.diag(1/x**2)

    x = newton_eq(f, grad_f, nabla_f, x_inner, A, b, MAXITERS=50, TOL=1e-5, alpha=0.000000001, beta=0.8)
    x_inner = x
    t *= mu

Iteration: 0, decrement: 10032.695561
Iteration: 1, decrement: 2624.523353
Iteration: 2, decrement: 698.389809
Iteration: 3, decrement: 125.625318
Iteration: 4, decrement: 24.939765
Iteration: 5, decrement: 8.829377
Iteration: 6, decrement: 5.958450
Iteration: 7, decrement: 2.692619
Iteration: 8, decrement: 2.033355
Iteration: 9, decrement: 1.934347
Iteration: 10, decrement: 0.186938
Iteration: 11, decrement: 0.005293
Iteration: 12, decrement: 0.000019
Iteration: 0, decrement: 2037.786831
Iteration: 1, decrement: 196.327503
Iteration: 2, decrement: 7.667300
Iteration: 3, decrement: 2.956204
Iteration: 4, decrement: 0.621368
Iteration: 5, decrement: 0.058405
Iteration: 6, decrement: 0.001358
Iteration: 0, decrement: 1898.415369
Iteration: 1, decrement: 186.971912
Iteration: 2, decrement: 1.096973
Iteration: 3, decrement: 0.135992
Iteration: 4, decrement: 0.015232
Iteration: 5, decrement: 0.000228
Iteration: 0, decrement: 1974.771067
Iteration: 1, decrement: 41.751164
Iteration: 2, decre

In [16]:
print(x_inner)

[6.64818222e-01 6.86979822e-01 2.32389720e-01 1.77816150e-10
 1.16113003e-10 2.07939480e-10 9.79795769e-11 1.41315498e+00
 1.79268202e-10 6.41541322e-10 5.05661047e-01 4.95011187e-08
 2.24561051e-10 3.17916675e-10 2.58169885e-10 2.60468514e-01
 1.03174632e+00 2.83142162e-01 2.00061977e-10 4.38678202e-10
 1.29842852e-10 1.76162065e+00 2.10855805e-10 6.38947249e-09
 2.54445681e-01 7.39687398e-01 4.36386017e-01 2.19749403e-01
 2.23610660e-01 8.27879999e-10 5.52666770e-01 8.05726809e-11
 9.39096575e-11 3.77229573e-01 5.72507576e-10 1.42440645e+00
 3.84036673e-01 7.87725040e-01 9.82858853e-02 1.42240708e+00
 1.11121251e+00 3.11734493e-10 6.56928767e-01 7.96900392e-01
 4.15724518e-01 9.14586636e-11 4.89886494e-01 3.70185154e-10
 2.51188814e+00 1.28391821e-01 1.69044374e-10 2.77112963e-10
 2.41558640e-08 3.65435910e-10 4.36372810e-01 1.15299537e-10
 1.30008322e+00 2.17078100e-01 2.19689078e-10 5.32204917e-10
 9.59839785e-11 7.26126115e-11 9.83511574e-02 6.50265021e-01
 1.02004600e-01 1.834612

We can see that this is not stable. We need to choose alpha carefully, otherwise it will not converge. The primal-dual method is more stable than barrier method.